### Тест ПАРСИНГА 

In [1]:
from collections import OrderedDict

In [2]:
def pars(rinFileName,csvFileName):
    import datetime
    from collections import OrderedDict
    rinFields = [] # list of data fields (L1, L2 etc) 
    numFields = 0  # numba of data fields
    # Open the rinex file and extract header data
    writeMode="w"
    rinFile = open(rinFileName, "r")
    csvFile = open(csvFileName, writeMode)
    
# Анализ заголовка RINEX - файла
    currentLine = rinFile.readline().rstrip('\n')

    while currentLine[60:].rstrip() != "END OF HEADER":
        #print(currentLine.rstrip('\n'))
        if currentLine[60:].rstrip() == "# / TYPES OF OBSERV": # Extract header data
            numFields = int(currentLine[1:6])
        
            #print("number of data columns ",numFields)
            firstLine='time,'+'N_sat,'
            for j in range(int(numFields/9)):
                #print("j=",j)
                for i in range(9):
                    rinFields.append(currentLine[(7+i*6):(12+i*6)].lstrip())
                    firstLine=firstLine+currentLine[(7+i*6):(12+i*6)].lstrip()+','
                currentLine = rinFile.readline().rstrip('\n')
            #print(numFields%9)            
            for i in range(numFields%9):
                rinFields.append(currentLine[(7+i*6):(12+i*6)].lstrip())
                firstLine=firstLine+currentLine[(7+i*6):(12+i*6)].lstrip()+','
                #print(i,currentLine)
        currentLine = rinFile.readline().rstrip('\n')
   
    #print(rinFields)
    # Удаляем последнюю запятую в списке
    firstLine=firstLine[:len(firstLine)-1]+""

    csvFile.write(firstLine+"\n")
    
    csvFile.close()
    print("END OF HEADER")

# СОЗДАЕМ СЛОВАРЬ ИЗ СПИСКА ПОЛЕЙ + ВРЕМЯ И N СПУТНИКА
    gpsRecord = OrderedDict(
      time = '0'
     
    )
    gpsRecord.update({'prn':'0'})
    # print(gpsRecord)
    
# create new dict from rinFields 
    d = OrderedDict.fromkeys(rinFields, 0)
# renew gpsrecord with new dict d 
    gpsRecord.update(d)
    #print(gpsRecord)
    # convert dict to list for writing list to csv file
    csvFields=list(gpsRecord)
    #csvFields=(gpsRecord.keys())
    #for i in 
    #    csvFields.append(gpsRecord)
    
# Open CSV file again and add data
    writeMode="a"
    csvFile = open(csvFileName, writeMode)
    print("ReOpenFile")
    # write first string
    # Scroll through looking for valid epochs
    currentLine = rinFile.readline().rstrip('\n')
    #print(gpsRecord)
    while currentLine != "":
        # current line starts new epoch?
        if (currentLine[27:30]).strip()  == "0":
            #print(currentLine)
            year = int(currentLine[1:3])
            if year < 80: 
                year = 2000 + year
            else: 
                1900 + year
            month = int(currentLine[4:6])
            day = int(currentLine[7:9])
            hour = int(currentLine[10:12].strip())
            mm = int(currentLine[13:15].strip())
            sec = float(currentLine[16:26].strip())
            
            # время 
            # gpsRecord["time"] = str(datetime.date(year, month, day).isoweekday() \
            #  % 7 * 24 * 3600 + hour * 3600 + mm * 60 + sec)
            #timef=datetime.date(year, month, day).isoweekday() \
            #  // 7  +
            timef=hour + mm/60. + sec/3600.
            gpsRecord["time"] = str('{:.3f}'.format(timef))   
            
# How many satellites in this epoch?
            numSats = int(currentLine[30:32]) # число спутников
            #print('numSats=',numSats)
            epochPRN = []
#           
# работает, если numSats>=12
            for j in range(int(numSats / 12)):
                for i in range(12):
                    epochPRN.append(currentLine[32+i*3:35+i*3])
                    #print(i,currentLine[32+i*3:35+i*3])
            #    читаем следующию строку для новых спутников > 12
            if numSats > 12:
                currentLine = rinFile.readline()
        # работает всегда
            for i in range(numSats % 12):

                epochPRN.append(currentLine[32+i*3:35+i*3])
                #print(i,currentLine[32+i*3:35+i*3])
            #print(numSats,epochPRN)
# parsing epoch
            
# reading next lines for each satellite                    
            for prn in epochPRN:
                currentLine = rinFile.readline().rstrip('\n')
                gpsRecord["prn"] = prn
                #print("prn1=",prn,currentLine)
            # работает, если numFields >=5
                for j in range(int(numFields / 5)):
                    
                    for i in range(5):
                        record = currentLine[i*16+1:i*16+14].lstrip()
                        if record == "":
                            gpsRecord[rinFields[j*5+i]] = "0"
                        else:
                            gpsRecord[rinFields[j*5+i]] = record
                if numFields>5:
                    currentLine = rinFile.readline().rstrip('\n')
                    #print("prn2=",prn,currentLine)
                for i in range(numFields % 5):
                    #currentLine = rinFile.readline()
                    record = currentLine[i*16+1:i*16+14].lstrip()
                    if record == "":
                        gpsRecord[rinFields[numFields//5*5+i]] = "0"
                    else:
                        gpsRecord[rinFields[numFields//5*5+i]] = record
            
            
                csvLine = ""
                
                for field in csvFields:
                    csvLine += gpsRecord[field] + ","
                csvFile.write(csvLine[:-1]+"\r\n")

        currentLine = rinFile.readline().rstrip('\n')


    # Close files 
    rinFile.close()
    csvFile.close()
    print('Parsing ',rinFileName,' end')


In [3]:
if __name__ == "__main__":
    
    rinFileName ="lama0100.18o" #"lama0101.19o" #"KLGD2860test.08o" #"lama0101.19o"
    csvFileName = "lama0100_18o.csv" #"lama0101.csv" #"KLGD2860test.csv" #"lama0101.csv"
    pars(rinFileName,csvFileName)

END OF HEADER
ReOpenFile
('Parsing ', 'lama0100.18o', ' end')


In [4]:
30/3600.


0.008333333333333333

In [5]:
11 % 12

11

In [6]:
linetest=" 0 "
newline=linetest.strip()
print(linetest,len(linetest))
print(newline,len(newline))

(' 0 ', 3)
('0', 1)
